## Params and loading packages

In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']



In [2]:

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Modules.Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 16)


### Till here

### Merge based on tissue and type. And annotate

In [3]:
if not os.path.exists('Results'):
    os.mkdir('Results')
    
if not os.path.exists('Results/merged'):
    os.mkdir('Results/merged')

In [4]:
all_peak_merge_files = []

for curr_tissue in tissues:
    curr_raw = glob.glob(data_folder + curr_tissue + '/*/')
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if not curr_type == 'ATAC':
            count = 1
            for f in  glob.glob(j + '*f04_peaks/merge_bg_2.peak'):
                all_peak_merge_files.append(f)

output_file = 'Results/merged/all_peaks_merged.tsv'                
merge_peaks(all_peak_merge_files,output_file,dist='given')


output_anno_file = 'Results/merged/all_peaks_merged_anno.tsv' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

mergePeaks -d given -venn Results/merged/all_peaks_merged.tsv_venn.tsv -matrix Results/merged/all_peaks_merged.tsv -strand /data/isshamie/TSS/Processed/BMDMwt/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Brain/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Brain/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Heart/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Kidney/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Kidney/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Liver/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Liver/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Lung/GRO/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Lung/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Muscle/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Pancreas/mSTART/f04_peaks/merge_bg_2.peak /data/isshamie/TSS/Processed/Spleen/GRO/f04_peaks/merge_bg_2.peak /data

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


annotatePeaks.pl Results/merged/all_peaks_merged.tsv /data/isshamie/genome/picr.fa -gff /data/isshamie/genome/updated_final_sort.gff3 > Results/merged/all_peaks_merged_anno.tsv


In [115]:
f_in = 'Results/merged/all_peaks_merged_anno.tsv'
ref_fa = '/data/isshamie/genome/picr.fa'
f_out = 'Results/merged/all_peaks_merged_anno.fa'
cmd = 'homerTools extract {f_in} {ref_fa} -fa > {f_out}'.format(f_in=f_in,ref_fa=ref_fa,f_out=f_out)
!{cmd}


	Extracting sequences from file: /data/isshamie/genome/picr.fa
	Looking for peak sequences in a single file (/data/isshamie/genome/picr.fa)
	Extracting 13371 sequences from picr_0
	Extracting 10521 sequences from picr_1
	Extracting 11440 sequences from picr_2
	Extracting 8592 sequences from picr_3
	Extracting 7741 sequences from picr_4
	Extracting 6873 sequences from picr_5
	Extracting 5218 sequences from picr_6
	Extracting 9335 sequences from picr_8
	Extracting 5709 sequences from picr_9
	Extracting 5089 sequences from picr_10
	Extracting 4571 sequences from picr_11
	Extracting 2503 sequences from picr_7b
	Extracting 4491 sequences from picr_12
	Extracting 5155 sequences from picr_13
	Extracting 4179 sequences from picr_14
	Extracting 3681 sequences from picr_15
	Extracting 4514 sequences from picr_16
	Extracting 2349 sequences from picr_17
	Extracting 2590 sequences from picr_18
	Extracting 3807 sequences from picr_19
	Extracting 4397 sequences from picr_20
	Extracting 2677 sequence

	Extracting 2 sequences from picr_397
	Extracting 1 sequences from picr_398
	Extracting 27 sequences from picr_400
	Extracting 1 sequences from picr_403
	Extracting 1 sequences from picr_404
	Extracting 1 sequences from picr_405
	Extracting 1 sequences from picr_409
	Extracting 1 sequences from picr_412
	Extracting 1 sequences from picr_413
	Extracting 1 sequences from picr_415
	Extracting 1 sequences from picr_416
	Extracting 1 sequences from picr_419
	Extracting 12 sequences from picr_420
	Extracting 1 sequences from picr_421
	Extracting 4 sequences from picr_430
	Extracting 1 sequences from picr_432
	Extracting 1 sequences from picr_433
	Extracting 1 sequences from picr_434
	Extracting 16 sequences from picr_435
	Extracting 2 sequences from picr_436
	Extracting 1 sequences from picr_438
	Extracting 1 sequences from picr_441
	Extracting 1 sequences from picr_443
	Extracting 1 sequences from picr_445
	Extracting 6 sequences from picr_454
	Extracting 4 sequences from picr_456
	Extracti

### Merge based on any sample file (so replicates are treated separately)
#### And annotate

In [5]:
all_peak_merge_files = []

for curr_tissue in tissues:
    curr_raw = glob.glob(data_folder + curr_tissue + '/*/')
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if not curr_type == 'ATAC':
            count = 1
            for f in  glob.glob(j + '*f04_peaks/trim*.peak'):
                all_peak_merge_files.append(f)

output_file = 'Results/merged/duplicatesSeparate_peaks_merged.tsv'                
merge_peaks(all_peak_merge_files,output_file,dist='given')
output_anno_file = 'Results/merged/duplicateSeparate_peaks_merged_anno.tsv' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

mergePeaks -d given -venn Results/merged/duplicatesSeparate_peaks_merged.tsv_venn.tsv -matrix Results/merged/duplicatesSeparate_peaks_merged.tsv -strand /data/isshamie/TSS/Processed/BMDMwt/GRO/f04_peaks/trim_CHBMDMwt_5GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001.fastq_and_trim_CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Brain/GRO/f04_peaks/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq_and_trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Brain/mSTART/f04_peaks/trim_CHBrain_mSTART_JHS1057_SD_ATTCCT_S79_L004_R1_001.fastq_and_trim_CHBrain_mSTARTinput_JHS1078_SD_ATTCCT_S100_L005_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Brain/mSTART/f04_peaks/trim_CHBrain_mSTART_JHS1016_SD_GTAGAG_S28_L002_R1_001.fastq_and_trim_CHBrain_mSTARTinput_JHS1078_SD_ATTCCT_S100_L005_R1_001.fastq_bg_2.peak /data/isshamie/TSS/Processed/Heart/mSTART/f04_peaks/trim_CHHeart_mSTART_JHS1056_SD_ATGAGC_S78_L004_R1_00

/home/isshamie/software/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


annotatePeaks.pl Results/merged/duplicatesSeparate_peaks_merged.tsv /data/isshamie/genome/picr.fa -gff /data/isshamie/genome/updated_final_sort.gff3 > Results/merged/duplicateSeparate_peaks_merged_anno.tsv


## Merge CHO all GRO and mSTART

In [10]:
all_peak_merge_files = []
curr_tissue ='CHO'
curr_raw = glob.glob(data_folder + curr_tissue + '/*/')
for j in curr_raw:
    curr_type = j.split('/')[-2]  
    if not curr_type == 'ATAC':
        count = 1
        for f in  glob.glob(j + '*f04_peaks/trim*.peak'):
            all_peak_merge_files.append(f)

output_file = 'Results/merged/CHO_samples_peaks_merged.tsv'                
merge_peaks(all_peak_merge_files,output_file,dist='given')
output_anno_file = 'Results/merged/CHO_samples_peaks_merged_anno.tsv' 
annotate_peaks(peak_file=output_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

mergePeaks -d given -venn Results/merged_samples/CHO_samples_peaks_merged.tsv_venn.tsv -matrix Results/merged_samples/CHO_samples_peaks_merged.tsv -strand /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD371_S10_R1_001_and_trim_CHO-GRO-SD369_S8_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD367_S6_R1_001_and_trim_CHO-GRO-SD370_S9_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD367_S6_R1_001_and_trim_CHO-GRO-SD369_S8_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/GRO/f04_peaks/trim_CHO-5GRO-SD371_S10_R1_001_and_trim_CHO-GRO-SD370_S9_R1_001_bg_2.peak /data/isshamie/CH_tissue_TSS/CHO/mSTART/f04_peaks/trim_CHO--mSTART-JHS823_S21_R1_001_and_trim_CHO--mSTART_input-JHS802_S11_R1_001_bg_2.peak > Results/merged_samples/CHO_samples_peaks_merged.tsv
annotatePeaks.pl Results/merged_samples/CHO_samples_peaks_merged.tsv /data/genome/hamster/picr/picr.fa -gff /data/genome/hamster/picr/updated_final_sort.gff3 > Results/m

## Use merged peaks to make histograms

In [1]:
anno_file = 'Results/merged/CHO_samples_peaks_merged_anno.tsv'
mRNA_peak_file= doc["mRNA_peak_file"] #"/data/isshamie/CH_tissue_TSS/mRNA.peak"
distance_from_promoter = 1000

out_folder = 'Results/merged'

In [ ]:
anno_peak(input_file,output_file)
peak_cov_hist(input_file,output_file,pc=0)
peak_cov_hist_mrna(input_file,output_file,pc=0)
peak_cov_hist_anno(input_file,output_file,pc=0)

## Convert to sequece

In [5]:
!pos2bed.pl Results/merged/all_peaks_merged_anno.tsv > Results/merged/all_peaks_merged.bed


	Converted 340547 peaks total



In [6]:
!pos2bed.pl Results/merged/duplicateSeparate_peaks_merged_anno.tsv > Results/merged/duplicateSeparate_peaks_merged_anno.bed


	Converted 470183 peaks total



In [14]:
f_in = 'Results/merged/duplicateSeparate_peaks_merged_anno.tsv'
ref_fa = '/data/isshamie/genome/picr.fa'
f_out = 'Results/merged/duplicateSeparate_peaks_merged_anno.fa'
cmd = 'homerTools extract {f_in} {ref_fa} -fa > {f_out}'.format(f_in=f_in,ref_fa=ref_fa,f_out=f_out)
!{cmd}


	Extracting sequences from file: /data/isshamie/genome/picr.fa
	Looking for peak sequences in a single file (/data/isshamie/genome/picr.fa)
	Extracting 18443 sequences from picr_0
	Extracting 14493 sequences from picr_1
	Extracting 15705 sequences from picr_2
	Extracting 12042 sequences from picr_3
	Extracting 10848 sequences from picr_4
	Extracting 9584 sequences from picr_5
	Extracting 7324 sequences from picr_6
	Extracting 12861 sequences from picr_8
	Extracting 7945 sequences from picr_9
	Extracting 7214 sequences from picr_10
	Extracting 6525 sequences from picr_11
	Extracting 3661 sequences from picr_7b
	Extracting 6327 sequences from picr_12
	Extracting 7128 sequences from picr_13
	Extracting 5878 sequences from picr_14
	Extracting 5086 sequences from picr_15
	Extracting 6151 sequences from picr_16
	Extracting 3343 sequences from picr_17
	Extracting 3745 sequences from picr_18
	Extracting 5325 sequences from picr_19
	Extracting 5976 sequences from picr_20
	Extracting 3803 seque

	Extracting 2 sequences from picr_436
	Extracting 4 sequences from picr_438
	Extracting 1 sequences from picr_441
	Extracting 1 sequences from picr_443
	Extracting 1 sequences from picr_445
	Extracting 1 sequences from picr_452
	Extracting 8 sequences from picr_454
	Extracting 1 sequences from picr_455
	Extracting 7 sequences from picr_456
	Extracting 5 sequences from picr_458
	Extracting 1 sequences from picr_459
	Extracting 20 sequences from picr_460
	Extracting 1 sequences from picr_461
	Extracting 2 sequences from picr_462
	Extracting 6 sequences from picr_463
	Extracting 4 sequences from picr_466
	Extracting 3 sequences from picr_469
	Extracting 1 sequences from picr_472
	Extracting 2 sequences from picr_474
	Extracting 8 sequences from picr_477
	Extracting 2 sequences from picr_478
	Extracting 1 sequences from picr_480
	Extracting 1 sequences from picr_481
	Extracting 3 sequences from picr_483
	Extracting 2 sequences from picr_484
	Extracting 1 sequences from picr_485
	Extracting

	Extracting 6 sequences from picr_1634
	Extracting 1 sequences from picr_1646
	Extracting 2 sequences from picr_1647
	Extracting 1 sequences from picr_1649
	Extracting 1 sequences from picr_1651
	Extracting 1 sequences from picr_1655
	Extracting 1 sequences from picr_1657
	Extracting 1 sequences from picr_1661
	Extracting 8 sequences from picr_1669
	Extracting 3 sequences from picr_1677
	Extracting 1 sequences from picr_1678
	Extracting 1 sequences from picr_1685
	Extracting 1 sequences from picr_1691
	Extracting 1 sequences from picr_1715
	Extracting 1 sequences from picr_1718
	Extracting 1 sequences from picr_1750
	Extracting 1 sequences from picr_1753
	Extracting 8 sequences from picr_1757
	Extracting 3 sequences from picr_1759
	Extracting 1 sequences from picr_1768
	Extracting 1 sequences from picr_1769
	Extracting 3 sequences from picr_1771
	Extracting 1 sequences from picr_1773
	Extracting 1 sequences from picr_1794
	Extracting 1 sequences from picr_1796
	Extracting 1 sequences f

# Merge all tag directories instead of peak files

In [7]:
curr_raw


['/data/isshamie/TSS/Processed/CHO/GRO/',
 '/data/isshamie/TSS/Processed/CHO/mSTART/',
 '/data/isshamie/TSS/Processed/CHO/ATAC/']

In [71]:
all_tags_start = []
for t in tissues:
    print(t)
    #curr = glob.glob(j + '*f03_tags/trim*.peak')
    curr_raw = glob.glob(data_folder + t + '/*/')
    #print(curr_raw)
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if curr_type == 'GRO':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = glob.glob(os.path.join(j,'f03_tags/*5GRO*'))
        elif curr_type == 'mSTART':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = np.array(glob.glob(os.path.join(j,'f03_tags/*')))
            if not len(new) == 0:
                new = (new[(np.array(map(lambda x: 'input' not in x,new)))])
        else:
            continue
        all_tags_start.extend(new)
        


curr_out = 'Results/merged/tags_TSS_merged'
cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=curr_out,files = ' '.join(all_tags_start))
print(cmd)
!{cmd}

BMDM1hKLA1
BMDMwt
Brain
FemaleReproductive
Heart
Intestine
Kidney
Liver
Lung
MiscOrgans
Muscle
Pancreas
Skin
Spleen
Total
CHO
makeTagDirectory Results/merged/tags_TSS_merged -single -d /data/isshamie/TSS/Processed/BMDMwt/GRO/f03_tags/trim_CHBMDMwt_5GRO_JHS1033_SD_TAGCTT_S48_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/GRO/f03_tags/trim_CHBrain_5GRO_JHS1036_SD_ATGTCA_S51_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/mSTART/f03_tags/trim_CHBrain_mSTART_JHS1016_SD_GTAGAG_S28_L002_R1_001.fastq /data/isshamie/TSS/Processed/Brain/mSTART/f03_tags/trim_CHBrain_mSTART_JHS1057_SD_ATTCCT_S79_L004_R1_001.fastq /data/isshamie/TSS/Processed/Heart/mSTART/f03_tags/trim_CHHeart_mSTART_JHS1013_SD_AGTTCC_S25_L002_R1_001.fastq /data/isshamie/TSS/Processed/Heart/mSTART/f03_tags/trim_CHHeart_mSTART_JHS1056_SD_ATGAGC_S78_L004_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/GRO/f03_tags/trim_CHKidney_5GRO_JHS1035_SD_AGTTCC_S50_L003_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/mSTART/f03_ta

	Adding tag directory /data/isshamie/TSS/Processed/Muscle/mSTART/f03_tags/trim_CHMuscle_mSTART_JHS1019_SD_GTGGCC_S31_L002_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Muscle/mSTART/f03_tags/trim_CHMuscle_mSTART_JHS1058_SD_CAAAAG_S80_L004_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Pancreas/mSTART/f03_tags/trim_CHPancreas_mSTART_JHS1014_SD_ATGTCA_S26_L002_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Pancreas/mSTART/f03_tags/trim_CHPancreas_mSTART_JHS1051_SD_GTTTCG_S73_L004_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Spleen/GRO/f03_tags/trim_CHBloondnegSpleen_5GRO_JHS1039_SD_GTCCGC_S54_L003_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Spleen/mSTART/f03_tags/trim_CHSpleen_mSTART_JHS1053_SD_GAGTGG_S75_L004_R1_001.fastq
	Adding tag directory /data/isshamie/TSS/Processed/Spleen/mSTART/f03_tags/trim_CHSpleednegBlood_mSTART_JHS1017_SD_GTCCGC_S29_L002_R1_001.fastq
	Adding tag directory /data/isshami

In [75]:
all_tags_input = []
for t in tissues:
    print(t)
    #curr = glob.glob(j + '*f03_tags/trim*.peak')
    curr_raw = glob.glob(data_folder + t + '/*/')
    #print(curr_raw)
    for j in curr_raw:
        curr_type = j.split('/')[-2]  
        if curr_type == 'GRO':
            #print(os.path.join(j,'f03_tags/*5GRO*'))
            new = glob.glob(os.path.join(j,'f03_tags/*_GRO*'))
            new.extend(glob.glob(os.path.join(j,'f03_tags/*-GRO*')))
        elif curr_type == 'mSTART':
            new = np.array(glob.glob(os.path.join(j,'f03_tags/*input*')))

        else:
            continue
        all_tags_input.extend(new)
        


curr_out = 'Results/merged/tags_input_merged'
cmd = 'makeTagDirectory {out_f} -single -d {files}'.format(out_f=curr_out,files = ' '.join(all_tags_input))
print(cmd)
!{cmd}

BMDM1hKLA1
BMDMwt
Brain
FemaleReproductive
Heart
Intestine
Kidney
Liver
Lung
MiscOrgans
Muscle
Pancreas
Skin
Spleen
Total
CHO
makeTagDirectory Results/merged/tags_input_merged -single -d /data/isshamie/TSS/Processed/BMDMwt/GRO/f03_tags/trim_CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/GRO/f03_tags/trim_CHBrain_GRO_JHS1029_SD_CACCGG_S44_L003_R1_001.fastq /data/isshamie/TSS/Processed/Brain/mSTART/f03_tags/trim_CHBrain_mSTARTinput_JHS1078_SD_ATTCCT_S100_L005_R1_001.fastq /data/isshamie/TSS/Processed/Heart/mSTART/f03_tags/trim_CHHeart_mSTARTinput_JHS1077_SD_ATGAGC_S99_L005_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/GRO/f03_tags/trim_CHKidney_GRO_JHS1028_SD_CAACTA_S43_L003_R1_001.fastq /data/isshamie/TSS/Processed/Kidney/mSTART/f03_tags/trim_CHKidney_mSTARTinput_JHS1080_SD_CAACTA_S102_L005_R1_001.fastq /data/isshamie/TSS/Processed/Liver/GRO/f03_tags/trim_CHLiver_GRO_JHS1031_SD_TAATCG_S46_L003_R1_001.fastq /data/isshamie/TSS/Processed/Liver/mS

## Find peaks and annotate

In [87]:
tag_dir = 'Results/merged/tags_TSS_merged'
control_dir = 'Results/merged/tags_input_merged'
peak_style = 'tss'
out_file = 'Results/merged/tags_merged.peak'
bg = '-F 2'
otherParams = [bg]

cmd = "findPeaks {tag} -style {style} -o {out} -i {control} -norm 1000000 {other}".format(
                tag=tag_dir,style=peak_style,out=out_file,control=control_dir,
                other=' '.join(otherParams))
print(cmd)
!{cmd}


findPeaks Results/merged/tags_TSS_merged -style tss -o Results/merged/tags_merged.peak -i Results/merged/tags_input_merged -norm 1000000 -F 2
	Fragment Length = 1
	Finding tags on separate strands: doubling effective genome size
	Finding peaks of size 150, no closer than 300
		Finding peaks on picr_0 (minCount=9.9), total tags positions = 1833029
		Finding peaks on picr_1 (minCount=9.9), total tags positions = 1187057
		Finding peaks on picr_10 (minCount=9.9), total tags positions = 638255
		Finding peaks on picr_100 (minCount=9.9), total tags positions = 145711
		Finding peaks on picr_1000 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_1001 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1002 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_1003 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1005 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_1006 (minCount=9.9), total tags positions = 134
		Find

		Finding peaks on picr_1110 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1111 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1112 (minCount=9.9), total tags positions = 183
		Finding peaks on picr_1113 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1114 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1115 (minCount=9.9), total tags positions = 30
		Finding peaks on picr_1116 (minCount=9.9), total tags positions = 47
		Finding peaks on picr_1118 (minCount=9.9), total tags positions = 26
		Finding peaks on picr_1119 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_112 (minCount=9.9), total tags positions = 75489
		Finding peaks on picr_1120 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_1121 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1123 (minCount=9.9), total tags positions = 214
		Finding peaks on picr_1124 (minCount=9.9), total tags positions = 2
		Finding 

		Finding peaks on picr_1230 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1231 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_1232 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_1233 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1234 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1237 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1238 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1239 (minCount=9.9), total tags positions = 37
		Finding peaks on picr_124 (minCount=9.9), total tags positions = 84838
		Finding peaks on picr_1240 (minCount=9.9), total tags positions = 101
		Finding peaks on picr_1241 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1242 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1243 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1244 (minCount=9.9), total tags positions = 1
		Finding peak

		Finding peaks on picr_1350 (minCount=9.9), total tags positions = 20
		Finding peaks on picr_1351 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1352 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_1353 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1354 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1355 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_1356 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1357 (minCount=9.9), total tags positions = 279
		Finding peaks on picr_1358 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_136 (minCount=9.9), total tags positions = 108847
		Finding peaks on picr_1360 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_1363 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_1364 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1366 (minCount=9.9), total tags positions = 9
		Finding pea

		Finding peaks on picr_1480 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_1481 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1482 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1483 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_1484 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1485 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_1486 (minCount=9.9), total tags positions = 12
		Finding peaks on picr_1487 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_1488 (minCount=9.9), total tags positions = 1
		Finding peaks on picr_1489 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_149 (minCount=9.9), total tags positions = 20557
		Finding peaks on picr_1490 (minCount=9.9), total tags positions = 823
		Finding peaks on picr_1491 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_1492 (minCount=9.9), total tags positions = 4
		Finding pea

...

		Finding peaks on picr_490 (minCount=9.9), total tags positions = 22
		Finding peaks on picr_491 (minCount=9.9), total tags positions = 46
		Finding peaks on picr_492 (minCount=9.9), total tags positions = 93
		Finding peaks on picr_493 (minCount=9.9), total tags positions = 27
		Finding peaks on picr_494 (minCount=9.9), total tags positions = 14
		Finding peaks on picr_495 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_496 (minCount=9.9), total tags positions = 34
		Finding peaks on picr_497 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_498 (minCount=9.9), total tags positions = 66
		Finding peaks on picr_499 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_5 (minCount=9.9), total tags positions = 801141
		Finding peaks on picr_50 (minCount=9.9), total tags positions = 440534
		Finding peaks on picr_500 (minCount=9.9), total tags positions = 15
		Finding peaks on picr_501 (minCount=9.9), total tags positions = 3
		Finding peaks on 

		Finding peaks on picr_60 (minCount=9.9), total tags positions = 82313
		Finding peaks on picr_600 (minCount=9.9), total tags positions = 16
		Finding peaks on picr_601 (minCount=9.9), total tags positions = 16
		Finding peaks on picr_602 (minCount=9.9), total tags positions = 13
		Finding peaks on picr_603 (minCount=9.9), total tags positions = 269
		Finding peaks on picr_604 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_605 (minCount=9.9), total tags positions = 329
		Finding peaks on picr_606 (minCount=9.9), total tags positions = 336
		Finding peaks on picr_607 (minCount=9.9), total tags positions = 26
		Finding peaks on picr_608 (minCount=9.9), total tags positions = 44
		Finding peaks on picr_609 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_61 (minCount=9.9), total tags positions = 422907
		Finding peaks on picr_610 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_611 (minCount=9.9), total tags positions = 4
		Finding peaks

		Finding peaks on picr_710 (minCount=9.9), total tags positions = 20
		Finding peaks on picr_711 (minCount=9.9), total tags positions = 16
		Finding peaks on picr_712 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_713 (minCount=9.9), total tags positions = 40
		Finding peaks on picr_714 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_715 (minCount=9.9), total tags positions = 28
		Finding peaks on picr_716 (minCount=9.9), total tags positions = 4
		Finding peaks on picr_717 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_718 (minCount=9.9), total tags positions = 13
		Finding peaks on picr_719 (minCount=9.9), total tags positions = 26
		Finding peaks on picr_72 (minCount=9.9), total tags positions = 115514
		Finding peaks on picr_720 (minCount=9.9), total tags positions = 29
		Finding peaks on picr_721 (minCount=9.9), total tags positions = 158
		Finding peaks on picr_722 (minCount=9.9), total tags positions = 47
		Finding peaks on p

		Finding peaks on picr_820 (minCount=9.9), total tags positions = 3
		Finding peaks on picr_821 (minCount=9.9), total tags positions = 87
		Finding peaks on picr_822 (minCount=9.9), total tags positions = 12
		Finding peaks on picr_823 (minCount=9.9), total tags positions = 6
		Finding peaks on picr_824 (minCount=9.9), total tags positions = 65
		Finding peaks on picr_825 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_826 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_827 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_828 (minCount=9.9), total tags positions = 10
		Finding peaks on picr_829 (minCount=9.9), total tags positions = 248
		Finding peaks on picr_83 (minCount=9.9), total tags positions = 220483
		Finding peaks on picr_830 (minCount=9.9), total tags positions = 8
		Finding peaks on picr_831 (minCount=9.9), total tags positions = 15
		Finding peaks on picr_832 (minCount=9.9), total tags positions = 4
		Finding peaks on pic

		Finding peaks on picr_940 (minCount=9.9), total tags positions = 43
		Finding peaks on picr_941 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_942 (minCount=9.9), total tags positions = 17
		Finding peaks on picr_943 (minCount=9.9), total tags positions = 17
		Finding peaks on picr_944 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_945 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_946 (minCount=9.9), total tags positions = 11
		Finding peaks on picr_947 (minCount=9.9), total tags positions = 34
		Finding peaks on picr_948 (minCount=9.9), total tags positions = 7
		Finding peaks on picr_95 (minCount=9.9), total tags positions = 87246
		Finding peaks on picr_950 (minCount=9.9), total tags positions = 9
		Finding peaks on picr_951 (minCount=9.9), total tags positions = 2
		Finding peaks on picr_952 (minCount=9.9), total tags positions = 5
		Finding peaks on picr_953 (minCount=9.9), total tags positions = 303
		Finding peaks on picr

## Compare with the merged peaks (duplicatesSeparate and overlap)

In [90]:
cmd = 'mergePeaks -d given -venn \
Results/merged/peaksMerged_tagsMerged.tsv_venn.tsv \
-matrix Results/merged/peaksMerged_tagsMerged.tsv \
-strand Results/merged/tags_merged.peak Results/merged/all_peaks_merged.tsv > \
Results/merged/peaksMerged_tagsMerged' 
!{cmd}

	Max distance to merge: direct overlap required (-d given)
	Merging peaks... 
	Duplicate peak name (Merged-picr_44-14150391-1) - this could potentially cause problems
		Sometimes unavoidable for BED/2DBED formats
		New name for this peak is Merged-picr_44-14150391-1--2
	Comparing Results/merged/tags_merged.peak (205822 total) and Results/merged/tags_merged.peak (205822 total)
	Comparing Results/merged/tags_merged.peak (205822 total) and Results/merged/all_peaks_merged.tsv (340547 total)
	Comparing Results/merged/all_peaks_merged.tsv (340547 total) and Results/merged/tags_merged.peak (205822 total)
	Comparing Results/merged/all_peaks_merged.tsv (340547 total) and Results/merged/all_peaks_merged.tsv (340547 total)



In [95]:
compare_merge = pd.read_csv('Results/merged/peaksMerged_tagsMerged.tsv_venn.tsv',sep='\t')
compare_merge

,Results/merged/tags_merged.peak,Results/merged/all_peaks_merged.tsv,Total,Name
0,NaN,X,174440,Results/merged/all_peaks_merged.tsv
1,X,NaN,39870,Results/merged/tags_merged.peak
2,X,X,165925,Results/merged/tags_merged.peak|Results/merged...


In [ ]:
ls Results/merged/tags_rna_merged/

In [103]:
tag_dir = 'Results/merged/tags_TSS_merged'
control_dir = 'Results/merged/tags_input_merged'
rna_f = 'Results/merged/tags_rna_merged'
out_f = 'Results/merged/tags_merged'
annotation = '/data/isshamie/genome/updated_final_sort.gtf'
cmd = 'findcsRNATSS.pl {start} -i {control} -rna {rna} -gtf {annotation} -genome ch -cpu 30 -o {out_f}'.format(
    start=tag_dir,
    control=control_dir,
    rna=rna_f,
    annotation=annotation,
    out_f=out_f)
print(cmd)
!{cmd}

findcsRNATSS.pl Results/merged/tags_TSS_merged -i Results/merged/tags_input_merged -rna Results/merged/tags_rna_merged -gtf /data/isshamie/genome/updated_final_sort.gtf -genome ch -cpu 30 -o Results/merged/tags_merged
	Fragment Length = 1
	Finding tags on separate strands: doubling effective genome size
	Finding peaks of size 150, no closer than 300
		Finding peaks on picr_0 (minCount=437.4), total tags positions = 1833029
		Finding peaks on picr_1 (minCount=437.4), total tags positions = 1187057
		Finding peaks on picr_10 (minCount=437.4), total tags positions = 638255
		Finding peaks on picr_100 (minCount=437.4), total tags positions = 145711
		Finding peaks on picr_1000 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_1001 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1002 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_1003 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1005 (minCount=437.4), total tags

		Finding peaks on picr_1110 (minCount=437.4), total tags positions = 6
		Finding peaks on picr_1111 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_1112 (minCount=437.4), total tags positions = 183
		Finding peaks on picr_1113 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_1114 (minCount=437.4), total tags positions = 6
		Finding peaks on picr_1115 (minCount=437.4), total tags positions = 30
		Finding peaks on picr_1116 (minCount=437.4), total tags positions = 47
		Finding peaks on picr_1118 (minCount=437.4), total tags positions = 26
		Finding peaks on picr_1119 (minCount=437.4), total tags positions = 7
		Finding peaks on picr_112 (minCount=437.4), total tags positions = 75489
		Finding peaks on picr_1120 (minCount=437.4), total tags positions = 5
		Finding peaks on picr_1121 (minCount=437.4), total tags positions = 1
		Finding peaks on picr_1123 (minCount=437.4), total tags positions = 214
		Finding peaks on picr_1124 (minCount=437.4), total t

		Finding peaks on picr_1230 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_1231 (minCount=437.4), total tags positions = 9
		Finding peaks on picr_1232 (minCount=437.4), total tags positions = 8
		Finding peaks on picr_1233 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1234 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_1237 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1238 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1239 (minCount=437.4), total tags positions = 37
		Finding peaks on picr_124 (minCount=437.4), total tags positions = 84838
		Finding peaks on picr_1240 (minCount=437.4), total tags positions = 101
		Finding peaks on picr_1241 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1242 (minCount=437.4), total tags positions = 1
		Finding peaks on picr_1243 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1244 (minCount=437.4), total tags 

		Finding peaks on picr_1350 (minCount=437.4), total tags positions = 20
		Finding peaks on picr_1351 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_1352 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_1353 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_1354 (minCount=437.4), total tags positions = 1
		Finding peaks on picr_1355 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_1356 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_1357 (minCount=437.4), total tags positions = 279
		Finding peaks on picr_1358 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_136 (minCount=437.4), total tags positions = 108847
		Finding peaks on picr_1360 (minCount=437.4), total tags positions = 9
		Finding peaks on picr_1363 (minCount=437.4), total tags positions = 8
		Finding peaks on picr_1364 (minCount=437.4), total tags positions = 6
		Finding peaks on picr_1366 (minCount=437.4), total tags

		Finding peaks on picr_1480 (minCount=437.4), total tags positions = 6
		Finding peaks on picr_1481 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_1482 (minCount=437.4), total tags positions = 1
		Finding peaks on picr_1483 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_1484 (minCount=437.4), total tags positions = 1
		Finding peaks on picr_1485 (minCount=437.4), total tags positions = 7
		Finding peaks on picr_1486 (minCount=437.4), total tags positions = 12
		Finding peaks on picr_1487 (minCount=437.4), total tags positions = 7
		Finding peaks on picr_1488 (minCount=437.4), total tags positions = 1
		Finding peaks on picr_1489 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_149 (minCount=437.4), total tags positions = 20557
		Finding peaks on picr_1490 (minCount=437.4), total tags positions = 823
		Finding peaks on picr_1491 (minCount=437.4), total tags positions = 5
		Finding peaks on picr_1492 (minCount=437.4), total tags

...

		Finding peaks on picr_770 (minCount=437.4), total tags positions = 16
		Finding peaks on picr_771 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_772 (minCount=437.4), total tags positions = 14
		Finding peaks on picr_773 (minCount=437.4), total tags positions = 6
		Finding peaks on picr_774 (minCount=437.4), total tags positions = 13
		Finding peaks on picr_775 (minCount=437.4), total tags positions = 6
		Finding peaks on picr_776 (minCount=437.4), total tags positions = 695
		Finding peaks on picr_777 (minCount=437.4), total tags positions = 28
		Finding peaks on picr_778 (minCount=437.4), total tags positions = 26
		Finding peaks on picr_779 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_78 (minCount=437.4), total tags positions = 140823
		Finding peaks on picr_780 (minCount=437.4), total tags positions = 7
		Finding peaks on picr_781 (minCount=437.4), total tags positions = 8
		Finding peaks on picr_782 (minCount=437.4), total tags position

		Finding peaks on picr_880 (minCount=437.4), total tags positions = 219
		Finding peaks on picr_881 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_882 (minCount=437.4), total tags positions = 19
		Finding peaks on picr_883 (minCount=437.4), total tags positions = 26
		Finding peaks on picr_884 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_885 (minCount=437.4), total tags positions = 25
		Finding peaks on picr_886 (minCount=437.4), total tags positions = 19
		Finding peaks on picr_887 (minCount=437.4), total tags positions = 2
		Finding peaks on picr_888 (minCount=437.4), total tags positions = 3
		Finding peaks on picr_889 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_89 (minCount=437.4), total tags positions = 194034
		Finding peaks on picr_890 (minCount=437.4), total tags positions = 14
		Finding peaks on picr_891 (minCount=437.4), total tags positions = 8
		Finding peaks on picr_892 (minCount=437.4), total tags positio

		Finding peaks on picr_990 (minCount=437.4), total tags positions = 17
		Finding peaks on picr_991 (minCount=437.4), total tags positions = 49
		Finding peaks on picr_992 (minCount=437.4), total tags positions = 21
		Finding peaks on picr_993 (minCount=437.4), total tags positions = 4
		Finding peaks on picr_995 (minCount=437.4), total tags positions = 5
		Finding peaks on picr_996 (minCount=437.4), total tags positions = 14
		Finding peaks on picr_997 (minCount=437.4), total tags positions = 35
		Finding peaks on picr_998 (minCount=437.4), total tags positions = 10
		Finding peaks on picr_999 (minCount=437.4), total tags positions = 9
		Tags Used for cluster (less clonal tags) = 291576068.0 / 291576068.0
	Expected tags per peak = 10.934102 (tbp = 0.145788)
	Finding tags on separate strands: doubling effective genome size
		Threshold	Peak Count	Expected Peak Count	FDR	Poisson
		38	44534.000	0.007	1.68e-07	1.41e-10
		37	44534.000	0.026	5.91e-07	4.93e-10
		36	44534.000	0.090	2.02e-06	1.

	Finding nearby peaks in 0.481623563523502.divergentTSS.txt
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 43708
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.301778172216476.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.301778172216476.tmp	23308	3.89	-69273.69

	Co-bound by 0 peaks: 20402
	Co-bound by 1 peaks: 23308 (max: 23308 effective total)
	Finding nearby peaks in 0.481623563523502.antisenseTSS.txt
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 86585
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.301778172216476.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.301778172216476.tmp	10705	2.76	-19440.69

	Co-bound by 0 peaks: 33005
	

	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.410145614369871.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.410145614369871.tmp	43708	4.76	0.00

	Co-bound by 0 peaks: 1
	Co-bound by 1 peaks: 43708 (max: 43708 effective total)
	Finding nearby peaks in 0.481623563523502.antisenseTSS.txt
	Peak/BED file conversion summary:
		BED/Header formatted lines: 0
		peakfile formatted lines: 86585
		Duplicated Peak IDs: 0
	Max distance to merge: direct overlap required (-d given)
	Calculating co-bound peaks relative to reference: 0.410145614369871.pos

	Comparing peaks: (peakfile, overlapping peaks, logRatio(obs/expected), logP)
		0.410145614369871.tmp	16265	3.29	-38783.03

	Co-bound by 0 peaks: 27444
	Co-bound by 1 peaks: 16265 (max: 16265 effective total)
	Finding Closest TSS...
	Processing custom annotation file...
	Features that will be considered:
		exon

	868909 lines contained useful fea

## Annotate Peaks

In [94]:
output_anno_file = 'Results/merged/tags_merged.anno' 
peak_file = 'Results/merged/tags_merged.peak'


annotate_peaks(peak_file=peak_file,output_file=output_anno_file ,ref_fa=ref_fa,annotation=annotation)

annotatePeaks.pl Results/merged/tags_merged.peak /data/isshamie/genome/picr.fa -gff /data/isshamie/genome/updated_final_sort.gff3 > Results/merged/tags_merged.anno
